# Example 3: Protein phosphorylation and diffusion in 3D cell geometry

Here, we implement the model of protein phosphorylation at the cell membrane and diffusion in the cytosol from [Meyers, Craig, and Odde 2006, Current Biology](https://doi.org/10.1016/j.cub.2006.07.056). It involves a single species $A$ that is phosphorylated at the cell membrane and dephosphorylated throughout the cytosol. For the full equations and more details, see the main example 3 file.

In [ ]:
import dolfin as d
import numpy as np
import pathlib
import logging

from smart import config, mesh, model, mesh_tools
from smart.units import unit
import sympy as sym
import gmsh  # must be imported before pyvista if dolfin is imported first

from smart import config, mesh, model, mesh_tools, visualization
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)

from matplotlib import pyplot as plt
import matplotlib.image as mpimg
logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)

Now, we define various units used in this problem.

In [ ]:
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
dimensionless = unit.dimensionless
D_unit = um**2 / sec
flux_unit = molecule / (um**2 * sec)
vol_unit = uM

Define compartments and species and create compartment and species containers. Note that the surface is subdivided into six surface compartments (see definition of cube mesh below).

In [ ]:
Cyto = Compartment("Cyto", 3, um, 1)
PMleft = Compartment("PMleft", 2, um, 10)
PMright = Compartment("PMright", 2, um, 11)
PMfront = Compartment("PMfront", 2, um, 12)
PMback = Compartment("PMback", 2, um, 13)
PMbottom = Compartment("PMbottom", 2, um, 14)
PMtop = Compartment("PMtop", 2, um, 15)
Aphos = Species("Aphos", 0.0, vol_unit, 10.0, D_unit, "Cyto")

cc = CompartmentContainer()
cc.add([PMleft, PMright, PMfront, PMback, PMtop, PMbottom, Cyto])
sc = SpeciesContainer()
sc.add([Aphos])

Define parameters and reactions and create parameter and reaction containers.

In [ ]:
Atot = Parameter("Atot", 1.0, vol_unit)
# Phosphorylation of Adephos at the PM
kkin = Parameter("kkin", 50.0, 1/sec)
curRadius = 2  # first radius value to test
# vol to surface area ratio of the cell (overwritten for each cell size)
VolSA = Parameter("VolSA", curRadius/3, um)
r1 = []
for PMsub in ["PMleft", "PMright", "PMfront", "PMback", "PMbottom", "PMtop"]:
    r1.append(Reaction(f"r1_{PMsub}", [], ["Aphos"], param_map={"kon": "kkin", "Atot": "Atot", "VolSA": "VolSA"},
              eqn_f_str="kon*VolSA*(Atot - Aphos)", species_map={"Aphos": "Aphos"}, explicit_restriction_to_domain=PMsub))
# Dephosphorylation of Aphos in the cytosol
kp = Parameter("kp", 10.0, 1/sec)
r2 = Reaction("r2", ["Aphos"], [], param_map={"kon": "kp"},
              eqn_f_str="kp*Aphos", species_map={"Aphos": "Aphos"})
pc = ParameterContainer()
pc.add([Atot, kkin, VolSA, kp])

rc = ReactionContainer()
rc.add([*r1, r2])

Define manufactured solution and associated reactions. To not make the choice of manufactured solution completely arbitrary, we have chosen it as a multiplication of three solutions to the PDE in 1D, then multiplied by some time-dependence governed by the time-scale of protein dephosphorylation in the cytosol.

In [ ]:
xSize, ySize, zSize = 2.0, 2.0, 2.0
x = sym.Symbol('x')
y = sym.Symbol('y')
z = sym.Symbol('z')
t = sym.Symbol('t')
k_kin = kkin.value*VolSA.value
k_p = kp.value
cT = Atot.value
D = Aphos.D
xFactor = xSize / sym.sqrt(3*D/k_p)
Ax = (k_kin*xSize/(2*D))*sym.exp(xFactor) / ((xFactor/2)*(sym.exp(xFactor)-1) + (k_kin*xSize/(2*D))*(1+sym.exp(xFactor)))
Bx = (k_kin*xSize/(2*D)) / ((xFactor/2)*(sym.exp(xFactor)-1) + (k_kin*xSize/(2*D))*(1+sym.exp(xFactor)))
yFactor = ySize / sym.sqrt(3*D/k_p)
Ay = (k_kin*ySize/(2*D))*sym.exp(yFactor) / ((yFactor/2)*(sym.exp(yFactor)-1) + (k_kin*ySize/(2*D))*(1+sym.exp(yFactor)))
By = (k_kin*ySize/(2*D)) / ((yFactor/2)*(sym.exp(yFactor)-1) + (k_kin*ySize/(2*D))*(1+sym.exp(yFactor)))
zFactor = zSize / sym.sqrt(3*D/k_p)
Az = (k_kin*zSize/(2*D))*sym.exp(zFactor) / ((zFactor/2)*(sym.exp(zFactor)-1) + (k_kin*zSize/(2*D))*(1+sym.exp(zFactor)))
Bz = (k_kin*zSize/(2*D)) / ((zFactor/2)*(sym.exp(zFactor)-1) + (k_kin*zSize/(2*D))*(1+sym.exp(zFactor)))
expFactor = sym.sqrt(k_p/(3*D))
Asol = cT * ((Ax*sym.exp(-expFactor*x) + Bx*sym.exp(expFactor*x)) * 
                (Ay*sym.exp(-expFactor*y) + By*sym.exp(expFactor*y)) * 
                (Az*sym.exp(-expFactor*z) + Bz*sym.exp(expFactor*z)) *
                (1 - sym.exp(-k_kin*t)))
f = sym.diff(Asol, t) - D*(sym.diff(sym.diff(Asol,x),x) + sym.diff(sym.diff(Asol,y),y) + sym.diff(sym.diff(Asol,z),z)) + k_p*Asol
f_parameter = Parameter.from_expression("f_parameter", str(f), uM/sec)#, use_preintegration=True)
f_reaction = Reaction("f_reaction", [], ["Aphos"], param_map={"k": "f_parameter"}, species_map={}, eqn_f_str="k")
pc.add([f_parameter])
rc.add([f_reaction])

gleft = -D*sym.diff(Asol,x) - k_kin*(Atot.value-Asol)
gleft.subs(x, 0.0)
gright = D*sym.diff(Asol,x) - k_kin*(Atot.value-Asol)
gright.subs(x, xSize)
gfront = -D*sym.diff(Asol,y) - k_kin*(Atot.value-Asol)
gfront.subs(y, 0.0)
gback = D*sym.diff(Asol,y) - k_kin*(Atot.value-Asol)
gback.subs(y, ySize)
gbottom = -D*sym.diff(Asol,z) - k_kin*(Atot.value-Asol)
gbottom.subs(z, 0.0)
gtop = D*sym.diff(Asol,z) - k_kin*(Atot.value-Asol)
gtop.subs(z, zSize)

g_params = []
g_reactions = []
PMsub_list = ["PMleft", "PMright", "PMfront", "PMback", "PMbottom", "PMtop"]
gExpr_list = [gleft,  gright,  gfront,  gback,  gbottom,  gtop]
for i in range(len(PMsub_list)):
    g_params.append(Parameter.from_expression(f"gParam_{PMsub_list[i]}", gExpr_list[i], uM*um/sec))#, use_preintegration=True))
    g_reactions.append(Reaction(f"g_{PMsub_list[i]}", [], ["Aphos"], param_map={"k": f"gParam_{PMsub_list[i]}"}, 
                                species_map={}, eqn_f_str="k", explicit_restriction_to_domain=PMsub_list[i]))
pc.add(g_params)
rc.add(g_reactions)

For this example, we use a cube mesh with different facet labels on each face. We generate a mesh at several different resolutions for convergence testing.

In [ ]:
# Set loglevel to warning in order not to pollute notebook output
logger.setLevel(logging.WARNING)

hrel = [0.2, 0.1, 0.05, 0.025, 0.0125]
L2_sums = []

for i in range(len(hrel)):

    elPerSide = int(1/hrel[i])
    domain = d.BoxMesh(d.Point(0.0, 0.0, 0.0), d.Point(xSize, ySize, zSize), elPerSide, elPerSide, elPerSide)
    cell_markers = d.MeshFunction("size_t", domain, 3, 1)
    facet_markers = d.MeshFunction("size_t", domain, 2, 0)
    for f in d.facets(domain):
        x, y, z = f.midpoint()[:]
        if np.isclose(x, 0.):
            facet_markers[f] = 10
        elif np.isclose(x, xSize):
            facet_markers[f] = 11
        elif np.isclose(y, 0.):
            facet_markers[f] = 12
        elif np.isclose(y, ySize):
            facet_markers[f] = 13
        elif np.isclose(z, 0.):
            facet_markers[f] = 14
        elif np.isclose(z, zSize):
            facet_markers[f] = 15

    # Write mesh and meshfunctions to file
    mesh_folder = pathlib.Path(f"mesh_hrel{hrel[i]:03f}/")
    mesh_folder.mkdir(exist_ok=True)
    mesh_file = mesh_folder / "DemoCube.h5"
    mesh_tools.write_mesh(domain, facet_markers, cell_markers, filename=mesh_file)
    # # Define parent mesh
    parent_mesh = mesh.ParentMesh(
        str(mesh_file),
        mesh_filetype="hdf5",
        name="parent_mesh",
    )

    config_cur = config.Config()
    model_cur = model.Model(pc, sc, cc, rc, config_cur, parent_mesh)
    config_cur.solver.update(
        {
            "final_t": 1,
            "initial_dt": 0.01,
            "time_precision": 6,
        }
    )
    config_cur.flags.update({"allow_unused_components": True, "print_verbose_info": False})
    model_cur.initialize()

    # Write initial condition(s) to file
    results = dict()
    result_folder = pathlib.Path(f"results_hrel{hrel[i]:03f}")
    result_folder.mkdir(exist_ok=True)
    for species_name, species in model_cur.sc.items:
        results[species_name] = d.XDMFFile(
            model_cur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
        )
        results[species_name].parameters["flush_output"] = True
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)

    model_cur.to_pickle('model_cur.pkl')

    # save integration measure and volume for computing average Aphos at each time step
    dx = d.Measure("dx", domain=model_cur.cc['Cyto'].dolfin_mesh)
    volume = d.assemble_mixed(1.0*dx)
    # Solve
    avg_Aphos = [Aphos.initial_condition]
    x, y, z = (sym.Symbol(f"x[{i}]") for i in range(3))
    from sympy.parsing.sympy_parser import parse_expr
    sym_expr = parse_expr(str(Asol)).subs({"x": x, "y": y, "z": z, "t": model_cur.t})
    Asol_expr = d.Expression(sym.printing.ccode(sym_expr), degree=3)
    avg_Asol = [d.assemble_mixed(Asol_expr*dx)/volume]
    L2_vec = [d.assemble_mixed((Asol_expr-model_cur.sc["Aphos"].u["u"])**2 *dx)]
    while True:
        # Solve the system
        model_cur.monolithic_solve()
        # Save results for post processing
        for species_name, species in model_cur.sc.items:
            results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
        # compute average Aphos concentration at each time step
        int_val = d.assemble_mixed(model_cur.sc['Aphos'].u['u']*dx)
        avg_Aphos.append(int_val / volume)
        sym_expr = parse_expr(str(Asol)).subs({"x": x, "y": y, "z": z, "t": model_cur.t})
        Asol_expr = d.Expression(sym.printing.ccode(sym_expr), degree=3)
        avg_Asol.append(d.assemble_mixed(Asol_expr*dx)/volume)
        L2_vec.append(d.assemble_mixed((Asol_expr-model_cur.sc["Aphos"].u["u"])**2 *dx))
        # End if we've passed the final time
        if model_cur.t >= model_cur.final_t:
            break
    plt.plot(model_cur.tvec, avg_Aphos)
    L2_sums.append(sum(L2_vec))
# visualization.plot(model_cur.sc['Aphos'].u['u'])
plt.plot(model_cur.tvec, avg_Asol, linestyle="dashed")
plt.xlabel('Time (s)')
plt.ylabel('Aphos concentration (μM)')

Plot L2 norm vs. mesh resolution.

In [ ]:
plt.loglog(hrel, L2_sums)